## Quest
- 업무분장(전처리, 모델학습).
- RecurrenceOfSurgery.csv 사용
- 목표변수 : 범주형, 설명변수 : 최소 6개 
- 서비스 대상과 목표 설명, 변수 선택 이유

## 분석 목적
- 서비스 제공 대상 : 제약 회사 
- 분석 목적 
    - 치료 기간 및 효과 분석: '입원기간'과 '통증기간(월)' 등의 데이터를 활용하여 환자들의 치료 기간과 효과를 분석합니다. 특정 치료 방법이나 제품이 어떤 기간 동안 환자의 상태 개선에 도움이 되는지를 확인하여 제품 효과를 검토할 수 있습니다.
    - 대상 환자 특성 파악: 선택된 컬럼을 기반으로 환자들의 연령, 성별, 체중 등 특성을 파악하여 제약회사의 대상 환자 프로필을 분석합니다. 이를 통해 어떤 연령층이나 성별에 해당하는 환자들에게 제품 또는 서비스를 맞춤화하여 제공할 수 있는지 확인합니다.
    
환자통증정도와 관련된 치료 기간 및 효과 분석, 환자 특성 변화 분석 등


### 변수 선택 
- 목표변수 : 환자통증정도
- 설명변수 : '입원기간', '통증기간(월)', '연령', '체중', 'Large Lymphocyte', '수술기법'

In [2]:
import pandas as pd

In [3]:
df_ROS = pd.read_csv('../dataset/RecurrenceOfSurgery.csv')
df_ROS[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [4]:
df_ROS.columns

Index(['Unnamed: 0', '환자ID', 'Large Lymphocyte', 'Location of herniation',
       'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수', '당뇨여부', '말초동맥질환여부', '빈혈여부',
       '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환', '암발병여부', '연령', '우울증여부', '입원기간',
       '입원일자', '종양진행여부', '직업', '체중', '퇴원일자', '헤모글로빈수치', '혈전합병증여부', '환자통증정도',
       '흡연여부', '통증기간(월)', '수술기법', '수술시간', '수술실패여부', '수술일자', '재발여부', '혈액형',
       '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도', 'Instability', 'MF + ES',
       'Modic change', 'PI', 'PT', 'Seg Angle(raw)', 'Vaccum disc', '골밀도',
       '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

In [5]:
df_ROS['환자통증정도'].value_counts()

7     885
8     410
9     172
2     130
10    107
1      52
5      46
3      44
6      40
4       8
Name: 환자통증정도, dtype: int64

In [6]:
df_ROS_extract = df_ROS[['환자통증정도','입원기간', '통증기간(월)', '연령', '체중', 'Large Lymphocyte', '수술기법']]

In [7]:
df_ROS_extract.info() # 설명변수 중 범주형 : 수술기법

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   환자통증정도            1894 non-null   int64  
 1   입원기간              1894 non-null   int64  
 2   통증기간(월)           1890 non-null   float64
 3   연령                1894 non-null   int64  
 4   체중                1894 non-null   float64
 5   Large Lymphocyte  1894 non-null   float64
 6   수술기법              1813 non-null   object 
dtypes: float64(3), int64(3), object(1)
memory usage: 103.7+ KB


### Pre-Processing (전처리과정)

#### Imputation (결측값 처리)

In [8]:
df_ROS_extract.isnull().sum()

환자통증정도               0
입원기간                 0
통증기간(월)              4
연령                   0
체중                   0
Large Lymphocyte     0
수술기법                81
dtype: int64

In [9]:
df_ROS_dropna = df_ROS_extract.dropna() # null값 제거한 df

In [10]:
df_ROS_dropna.isnull().sum()

환자통증정도              0
입원기간                0
통증기간(월)             0
연령                  0
체중                  0
Large Lymphocyte    0
수술기법                0
dtype: int64

#### Encoding with One Hot Encoding
범주형 데이터를 연속형 숫자 데이터로 변환
범주형 항목만큼 벡터화

In [11]:
df_ROS_dropna['수술기법'].value_counts()

TELD    1669
IELD     140
Name: 수술기법, dtype: int64

In [12]:
from sklearn.preprocessing import OneHotEncoder
oneHotEncoder = OneHotEncoder() #인스턴스화 
oneHotEncoder.fit(df_ROS_dropna[['수술기법']])# 학습
oneHotEncoder.categories_

[array(['IELD', 'TELD'], dtype=object)]

In [13]:
encoded_data = oneHotEncoder.transform(df_ROS_dropna[['수술기법']]).toarray()
encoded_data

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])

In [14]:
df_encoded_data = pd.DataFrame(data=encoded_data, columns=oneHotEncoder.get_feature_names_out(['수술기법']))
df_encoded_data[:3]

,수술기법_IELD,수술기법_TELD
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0


In [15]:
df_encoded_data.index, df_encoded_data.shape

(RangeIndex(start=0, stop=1809, step=1), (1809, 2))

In [16]:
df_ROS_dropna.index, df_ROS_dropna.shape ## 인덱스의 갯수가 맞음

(Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
             ...
             1843, 1845, 1849, 1865, 1866, 1870, 1872, 1874, 1879, 1891],
            dtype='int64', length=1809),
 (1809, 7))

#### concat(병합)

In [17]:
df_ROS_concat = pd.concat([df_ROS_dropna.reset_index(drop=True), df_encoded_data.reset_index(drop=True)], axis=1 )
df_ROS_concat

,환자통증정도,입원기간,통증기간(월),연령,체중,Large Lymphocyte,수술기법,수술기법_IELD,수술기법_TELD
0,10,2,1.0,66,60.3,22.8,TELD,0.0,1.0
1,10,1,1.0,47,71.7,44.9,TELD,0.0,1.0
2,7,1,1.0,39,77.1,53.0,TELD,0.0,1.0
3,7,1,2.0,40,74.2,54.7,TELD,0.0,1.0
4,7,2,1.0,42,80.7,53.8,TELD,0.0,1.0
...,...,...,...,...,...,...,...,...,...
1804,7,12,24.0,29,52.0,32.4,TELD,0.0,1.0
1805,8,2,36.0,21,64.0,42.4,TELD,0.0,1.0
1806,6,3,6.0,43,60.0,30.8,TELD,0.0,1.0
1807,8,3,4.0,20,70.0,5.4,TELD,0.0,1.0


In [18]:
df_ROS_concat.columns

Index(['환자통증정도', '입원기간', '통증기간(월)', '연령', '체중', 'Large Lymphocyte', '수술기법',
       '수술기법_IELD', '수술기법_TELD'],
      dtype='object')

#### Scaling - MinMaxScaler

In [19]:
target = df_ROS_concat['환자통증정도']
features = df_ROS_concat.drop(columns=['환자통증정도','수술기법'])

In [20]:
features.columns

Index(['입원기간', '통증기간(월)', '연령', '체중', 'Large Lymphocyte', '수술기법_IELD',
       '수술기법_TELD'],
      dtype='object')

In [21]:
from sklearn.preprocessing import MinMaxScaler
minMaxScaler= MinMaxScaler()
features = minMaxScaler.fit_transform(features)

#### Imbalanced Data Sampling
-  Under Sampling : Tomek's Link 

In [23]:
# from imblearn.under_sampling import TomekLinks
# from sklearn.datasets import make_classification

ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (C:\Users\user\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py)

In [24]:
features, target = make_classification(n_classes=2, class_sep=2,
                    weights=[0.3, 0.7], n_informative=3, n_redundant=1, flip_y=0,
                    n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)

NameError: name 'make_classification' is not defined

In [ ]:
features.shape, target.shape

In [25]:
from collections import Counter
Counter(target)

Counter({10: 107,
         7: 852,
         8: 386,
         9: 163,
         2: 125,
         1: 51,
         6: 37,
         5: 40,
         3: 44,
         4: 4})

In [26]:
tomekLinks = TomekLinks() #인스턴스화
features_resaple, target_resample = tomekLinks.fit_resample(features, target) #학습

NameError: name 'TomekLinks' is not defined

In [ ]:
features_resaple.shape, target_resample.shape

In [ ]:
Counter(target_resample)

#### 결측값을 채워놓기 위한 모델 학습 

In [27]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier() #인스턴스화 
model.fit(features, target) #모델 훈련

DecisionTreeClassifier()

In [29]:
from sklearn.tree import DecisionTreeRegressor
# 결측치를 예측할 모델을 인스턴스화
model = DecisionTreeRegressor()

In [40]:
# 결측치가 없는 행을 기반으로 모델 학습
X_train = df_ROS_concat
y_train = X_train['통증기간(월)']  # 예측 대상 컬럼
X_train = X_train.drop(columns=['통증기간(월)','수술기법'], axis=1)  # 예측에 사용할 특성 선택
model.fit(X_train, y_train)

DecisionTreeRegressor()

#### 결측값 채우기

In [ ]:
def return_data(row):
    result = model(row)
    return result

In [43]:
def predict_missing_duration(row):
    if pd.isnull(row['통증기간(월)']):
        # 결측치를 예측하기 위해 사용할 특성 선택 (통증기간(월) 제외)
        features = row.drop(['통증기간(월)'])
        # 모델을 사용하여 결측치 예측
        predicted_duration = model.predict([features])[0]
        return predicted_duration
    else:
        # 값이 있는 경우 그대로 반환
        return row['통증기간(월)']

In [44]:
df_ROS_extract['통증기간(월)_filled'] = df_ROS_extract.apply(predict_missing_duration, axis=1)

C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


ValueError: could not convert string to float: 'TELD'

In [ ]:
df_ROS_dropna.isnull().sum()

In [ ]:
df_ROS_dropna['통증기간(월)_return']